In [2]:
import scipy
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import scipy.optimize as spo
import scipy.constants as const
from scipy.spatial.transform import Rotation as R

# Constants

In [ ]:
# Valor de bohr en metros
bohr_factor = const.value('Bohr radius')
# Factor de conversión de angstrom a bohr
angstrom_to_bohr = const.angstrom / bohr_factor # 1.8897261246257702
hartree_to_kcal = 627.509
kJ_to_kcal = 1 / 4.184
hartree_to_kJ = 2625.5

# Data  PF6 - CHN

In [6]:
energies_kcal = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\total_energies_kcal.npy')
distances_angstrom_137x154 = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\distances_angstrom.npy')
charges_chn = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\charges_chn.npy')
charges_pf6 = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\charges_pf6.npy')

## Aqui falta un set de datos de PF6 y CHN que sea con datos de 80 muestras, no recuerdo ni como las saque

# DATA PF6 - PF6

In [6]:
energies_pf6_pf6_kcal = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\pf6_pf6_energies.npy')
distanceswcharge_pf6_pf6_angstrom = np.load(r'C:\Users\migue\OneDrive\Desktop\QM_Project\Parametrization\pf6_pf6_distanceswcharge.npy') 
# distances matrix have for each energy: distance between molecules, index of combination (f with f, f with p, p with p)
# and charge multiplication, in that order. 

# <span style="color:Green"> Dispersion coefficients of pf6 from alston </span>

In [ ]:
# C_6 coeficients - units ht*bohr**6
cpp = 30.61343
cfp = 18.70394
cff = 11.50171

# Functions

# dispersion coeficient by itself should do a good value because we got it from a realiable source, just need scaling.
# electrostatic term by itself should have good values because we got it from good sources. 

### Step 1: get the scaling factor again, just the scaling factor to maintain the presition of the dispersion part in the molecules. 

because all of the variables of williams potential and openmm utilize kJ/mol it is a good idea to begin with all this changes, only is not necesary in the electrostatic part because the charges can be transformed in the end without affecting the final results. 

### Step 2: With fixed dispersion coefficients, we can fit the exponential and multiplicative factors.

In [ ]:

def energy_williams_pf6_pf6(combination_matrix):
    combination_matrix = np.array(combination_matrix)
    energies = []
    for i in range(combination_matrix.shape[0]):
        sum = 0
        for j in range(combination_matrix.shape[2]):
            exp_term = (combination_matrix[i, 2, j])*(np.exp((-combination_matrix[i, 3, j])*(combination_matrix[i, 0, j])))
            dis_ind_term = combination_matrix[i, 1, j]/((combination_matrix[i, 0, j])**6)
            elst_term = (combination_matrix[i, 4, j])/(combination_matrix[i, 0, j]*angstrom_to_bohr)
            sum = sum + (exp_term*kJ_to_kcal) - (dis_ind_term*kJ_to_kcal) + (elst_term*hartree_to_kcal)
        energies.append(sum)
    return energies

In [ ]:
def energy_williams_pf6_pf6(distances, charges,):